In [36]:
import pandas as pd
import geopandas as gpd

In [37]:
location = pd.read_csv('regression_data.csv')
flood_points = pd.read_csv('cleaned/final.csv')

In [38]:
geocoded_data = gpd.GeoDataFrame(location, geometry = gpd.points_from_xy(location.longitude, location.latitude), crs = "EPSG:4326").to_crs('EPSG:3857')
flood_points_data = gpd.GeoDataFrame(flood_points, geometry = gpd.points_from_xy(flood_points.longitude, flood_points.latitude), crs = "EPSG:4326").to_crs('EPSG:3857')

In [39]:
flood_points_data.head()

,date,negeri,code,daerah,kawasan_banjir,latitude,longitude,kedalaman_banjir_max,day,month,year,line_check,geometry
0,29/4/2020,Sarawak,SRW,TANGKAK,2. Kg Parit Bengkok,1.5203,103.9244,0.5,29,4,2020,1,POINT (11568811.289 169258.885)
1,10/5/2020,Sarawak,SRW,KLUANG,Kg Bentong,2.0475,103.2973,0.5,10,5,2020,1,POINT (11499002.836 227975.185)
2,10/5/2020,Sarawak,SRW,KLUANG,Jalan Batu Pahat (Petron Tmn Berlian),2.0173,103.2960,0.5,10,5,2020,1,POINT (11498858.121 224611.220)
3,10/5/2020,Sarawak,SRW,KLUANG,Taman Intan,2.0147,103.3009,0.5,10,5,2020,1,POINT (11499403.586 224321.610)
4,10/5/2020,Sarawak,SRW,KLUANG,Taman Emas,2.0205,103.2986,0.5,10,5,2020,1,POINT (11499147.552 224967.663)


In [40]:
# Select a random location
random_location = geocoded_data.iloc[360]

# Measure distance from release to each station
distances = flood_points_data.geometry.distance(random_location.geometry)
distances

0       685886.758825
1       597715.537230
2       600426.839376
3       600971.052340
4       600291.870584
            ...      
6950    428962.466124
6951    367725.272507
6952    367868.012406
6953    414189.826853
6954    366620.377337
Length: 6955, dtype: float64

In [41]:
print("Mean distance: {}".format(distances.mean()))
print("Nearest flood point: {}".format(distances.min()))

# find that nearest flood point
print(flood_points_data.iloc[distances.idxmin()][["kawasan_banjir", "latitude", "longitude"]])

Mean distance: inf
Nearest flood point: 5597.966928358064
kawasan_banjir    Kampung Felcra Ulu\nPauh
latitude                            6.4527
longitude                         100.3328
Name: 1148, dtype: object


In [43]:
# write a function to compute for the minimum distances from any flood point
random_location = geocoded_data.iloc[360]
distances = flood_points_data.geometry.distance(random_location.geometry)
distances.min()

for i in range(0, len(geocoded_data)):
    distances = flood_points_data.geometry.distance(geocoded_data.iloc[i].geometry)
    geocoded_data.loc[i, 'distance'] = distances.min()
    geocoded_data.loc[i, 'kawasan_banjir'] = flood_points_data.iloc[distances.idxmin()]["kawasan_banjir"]
    geocoded_data.loc[i, 'latitude_kb'] = flood_points_data.iloc[distances.idxmin()]["latitude"]
    geocoded_data.loc[i, 'longitude_kb'] = flood_points_data.iloc[distances.idxmin()]["longitude"]
    geocoded_data.loc[i, 'kedalaman_banjir'] = flood_points_data.iloc[distances.idxmin()]["kedalaman_banjir_max"]
    geocoded_data.loc[i, 'month_kb'] = flood_points_data.iloc[distances.idxmin()]["month"]

In [44]:
geocoded_data.head()

,latitude,longitude,region,state,postcode,geometry,flood_risk,distance,kawasan_banjir,latitude_kb,longitude_kb,kedalaman_banjir,month_kb
0,6.402858,100.098777,Kangar,Perlis,1000,POINT (11142944.907 714251.095),False,4245.737601,Kampung Nelayan,6.4040,100.1369,0.6,4.0
1,6.365790,100.188445,Kangar,Perlis,1000,POINT (11152926.670 710098.960),False,4532.533863,Kampung Sentua,6.4051,100.1981,0.2,10.0
2,6.360280,100.212395,Kangar,Perlis,1000,POINT (11155592.794 709481.714),False,5266.664356,Kampung Sentua,6.4051,100.1981,0.2,10.0
3,6.557335,100.314105,Kangar,Perlis,1000,POINT (11166915.133 731557.973),False,8652.136771,Taman Beseri Jaya,6.5590,100.2364,0.6,5.0
4,6.395771,100.230483,Kangar,Perlis,1000,POINT (11157606.330 713457.232),False,3753.265280,Kampung Sentua,6.4051,100.1981,0.2,10.0


In [45]:
# update the file once done
geocoded_data.to_csv('regression_data.csv', index = False)

In [46]:
geocoded_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19824 entries, 0 to 19823
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   latitude          19824 non-null  float64 
 1   longitude         19824 non-null  float64 
 2   region            19824 non-null  object  
 3   state             19824 non-null  object  
 4   postcode          19824 non-null  int64   
 5   geometry          19824 non-null  geometry
 6   flood_risk        19824 non-null  bool    
 7   distance          19824 non-null  float64 
 8   kawasan_banjir    19824 non-null  object  
 9   latitude_kb       19824 non-null  float64 
 10  longitude_kb      19824 non-null  float64 
 11  kedalaman_banjir  19824 non-null  float64 
 12  month_kb          19824 non-null  float64 
dtypes: bool(1), float64(7), geometry(1), int64(1), object(3)
memory usage: 1.8+ MB


In [47]:
geocoded_data.describe()

,latitude,longitude,postcode,distance,latitude_kb,longitude_kb,kedalaman_banjir,month_kb
count,19824.000000,19824.000000,19824.000000,19824.000000,19824.000000,19824.000000,19824.000000,19824.000000
mean,4.164278,104.262531,48368.675141,4071.710091,4.165323,104.262720,0.574154,6.892151
std,1.504161,5.413517,29732.309123,4790.504712,1.501299,5.413495,0.535538,3.837407
min,1.045992,99.662046,1000.000000,19.253343,0.987600,99.712200,-5.000000,1.000000
25%,3.078602,101.263753,22010.000000,1313.426109,3.078250,101.310200,0.300000,4.000000
50%,4.017306,101.890486,46630.000000,2683.402344,4.018800,101.886200,0.450000,7.000000
75%,5.487689,103.339178,75904.000000,4827.023973,5.467300,103.331900,0.600000,11.000000
max,6.911623,118.651558,98857.000000,66853.687820,6.925100,118.421800,6.000000,12.000000
